In [309]:
import data_load as dl
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Model
from keras.layers import Input, Conv2D, ZeroPadding1D, Conv2DTranspose, UpSampling2D, Dense
from keras.layers.pooling import MaxPooling2D, MaxPooling1D
from keras import optimizers
import sklearn
import tensorflow as tf


%matplotlib inline

In [2]:
path = r"C:\Users\Patrick\Datasets\sr\SR"
trial_paths = dl.combine_data(path)
trials = dl.generate_trial_objects(trial_paths)
processed_trials = dl.process_trial_data(trials)

C:\Users\Patrick\Anaconda3\lib\site-packages\openpyxl\reader\worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
#Smallest motion sample is 17559
# All G samples are 3030 long
# Take 2880 points from middle of 3030
# 3030 - 2880 = 150
#150/2 = 75
#Start each (Gs and downsampled motion on point 75) then sample out 2880

In [256]:
def synch_streams(G3, G4, motion):
    #G2 = G2[75:-75]
    G3 = G3[75:-75]
    G4 = G4[75:-75]
    #G5 = G5[75:-75]
    
    new_motion = motion[0:-1:6]
    new_motion = motion[75:(75+2880)]
    
    return G3, G4, new_motion

In [257]:
def generate_segments(G3, G4, motion):
    full_run_length = 2880
    num_segments = 200
    indices = np.random.randint(0,(full_run_length - 112), size=(num_segments))
    
    images_out = []
    motion_out = []
    
    for i in indices:
        #G2_seg = G2[i:(i+112)]/np.amax(G2[i:(i+112)])
        G3_seg = G3[i:(i+112)]/np.amax(G3[i:(i+112)])
        G4_seg = G4[i:(i+112)]/np.amax(G4[i:(i+112)])
        #G5_seg = G5[i:(i+112)]/np.amax(G5[i:(i+112)])
        motion_seg = motion[i:(i+112)]/np.amax(motion[i:(i+112)])
        
        new_image = np.array([[G3_seg],[G4_seg]])
        
        images_out.append(new_image)
        
        motion_out.append(np.asarray(motion_seg))
        
    

    return images_out, motion_out

In [287]:
X = []
y = []

for i in range(25):
    trial = processed_trials[i]
    G3_temp, G4_temp, motion_temp = synch_streams(trial.G3_AVG, trial.G4_AVG, trial.Motion_Conv)
    images, motion = generate_segments(G3_temp, G4_temp, motion_temp)
    
    for image in images:
        new_image=np.array(image)
        new_image.reshape(1,112,2)
        X.append(new_image)
    
    for entry in motion:
        y.append(np.array(entry))
        
X = np.array(X).reshape(5000,1,112,2)
y = np.array(y)

In [288]:
# trial = processed_trials[5]
# G3_temp, G4_temp, motion_temp = synch_streams(trial.G3_AVG, trial.G4_AVG, trial.Motion_Conv)
# images, motion = generate_segments(G3_temp, G4_temp, motion_temp)

# X = []
# y = []

# for image in images:
#     new_image=np.array(image)
#     new_image.reshape(1,112,2)
#     X.append(new_image)
    
# for entry in motion:
#     y.append(np.array(entry))
    


In [289]:
X.shape

(5000, 1, 112, 2)

In [290]:
y.shape

(5000, 112)

In [291]:
#X = np.expand_dims(X, axis=2)
#X = np.expand_dims(X, axis=3)

y = np.expand_dims(y, axis=1)
y.reshape(5000,1,112)
y = np.expand_dims(y, axis=3)

In [292]:
X.shape

(5000, 1, 112, 2)

In [293]:
y.shape

(5000, 1, 112, 1)

In [294]:
# model = Sequential()
# model.add(Conv2D(64, (2,2), padding='same', activation='relu', input_shape=(100,1,2)))
# #model.add(MaxPooling2D(2))

# # model.add(Conv2D(24, (2,2), padding='same', activation='relu'))
# # model.add(MaxPooling2D(1,2))

# # model.add(Conv2DTranspose(1, (1,1), strides=(1,1), activation='relu'))
# # model.add(UpSampling2D(size=(1,2)))

# #model.add(Conv2DTranspose(1, (1,1), strides=(1,1), activation='relu'))
# #model.add(UpSampling2D(size=(2,1)))


# model.compile(optimizer='adam', loss = 'mean_squared_error', metrics = ['accuracy'])

In [312]:
inputs = Input(shape=(1,112,2), name="main_input")

x1 = Conv2D(64, kernel_size=(1,3), padding="same")(inputs)
x1 = Conv2D(64, kernel_size=(1,3),padding="same")(x1)
x1 = MaxPooling2D(pool_size=(1,2), strides=(2))(x1)

x2 = Conv2D(128, kernel_size=(1,3),padding="same")(x1)
x2 = Conv2D(128, kernel_size=(1,3),padding="same")(x2)
x2 = MaxPooling2D(pool_size=(1,2), strides=(2))(x2)

x3 = Conv2D(256, kernel_size=(1,3),padding="same")(x2)
x3 = Conv2D(256, kernel_size=(1,3),padding="same")(x3)
x3 = MaxPooling2D(pool_size=(1,2), strides=(2))(x3)

x4 = Conv2D(512, kernel_size=(1,3),padding="same")(x3)
x4 = Conv2D(512, kernel_size=(1,3),padding="same")(x4)
x4 = MaxPooling2D(pool_size=(1,2), strides=(2))(x4)

x5 = Conv2D(1024, kernel_size=(1,7))(x4)

x6 = Dense(1024)(x5)

#Presently doing unpooling in hacked way (No filtering for max value, naive upsampling)
# https://arxiv.org/pdf/1311.2901v3.pdf
# https://stackoverflow.com/questions/44991470/using-tensorflow-layers-in-keras

#Need to create custom keras layer to take mask from MaxPooling2D (similar to TF's max_pool_with_argmax)
#mask is used in unpooling layer

x7 = Conv2DTranspose(512, kernel_size=(1,7), padding='valid')(x6)
x7 = UpSampling2D((1,2))(x7)

x8 = Conv2DTranspose(512, kernel_size=(1,3), padding='same')(x7)
x8 = Conv2DTranspose(512, kernel_size=(1,3), padding='same')(x8)
x8 = UpSampling2D((1,2))(x8)

x9 = Conv2DTranspose(512, kernel_size=(1,3), padding='same')(x8)
x9 = Conv2DTranspose(256, kernel_size=(1,3), padding='same')(x9)
x9 = UpSampling2D((1,2))(x9)

x10 = Conv2DTranspose(256, kernel_size=(1,3), padding='same')(x9)
x10 = Conv2DTranspose(128, kernel_size=(1,3), padding='same')(x10)
x10 = UpSampling2D((1,2))(x10)

x11 = Conv2DTranspose(128, kernel_size=(1,3), padding='same')(x10)
x11 = Conv2DTranspose(64, kernel_size=(1,3), padding='same')(x11)

x11 = Conv2DTranspose(2, kernel_size=(1,3), padding='same')(x11)
x12 = Conv2DTranspose(1, kernel_size=(1,3), padding='same')(x11)

model2 = Model(inputs=inputs, outputs=x12)

sgd = optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0005)

model2.compile(optimizer=sgd, loss = 'mean_squared_error', metrics = ['mse'])

In [314]:
#model1.fit(X,y, epochs=10)

In [304]:
np.save("X.npy",X)

In [305]:
np.save("y.npy",y)